In [1]:
#import the requires libaries
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import dlib
import tensorflow as tf
import math
from PIL import Image as im

In [3]:
#import the model we trained and saved in the other notebook
new_model=tf.keras.models.load_model('eye_track.h5')

2022-08-23 16:23:35.132282: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [4]:
#prepared the image so that it can be understood by the model
def prepare(filepath):
    img_array=cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    new_array=cv2.resize(img_array,(400,600))
    return new_array.reshape(-1,200,200,3)

#turns on the webcam to capture the face
cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        
        #draws green lines across the eyes to identify them
        landmarks = predictor(gray, face)
        Lleft_point = (landmarks.part(36).x, landmarks.part(36).y)
        Lright_point = (landmarks.part(39).x, landmarks.part(39).y)
        Lcenter_top = midpoint(landmarks.part(37), landmarks.part(38))
        Lcenter_bottom = midpoint(landmarks.part(41), landmarks.part(40))

        horL_line = cv2.line(frame, Lleft_point, Lright_point, (0, 255, 0), 2)
        verL_line = cv2.line(frame, Lcenter_top, Lcenter_bottom, (0, 255, 0), 2)
        
        Rleft_point = (landmarks.part(42).x, landmarks.part(42).y)
        Rright_point = (landmarks.part(45).x, landmarks.part(45).y)
        Rcenter_top = midpoint(landmarks.part(43), landmarks.part(44))
        Rcenter_bottom = midpoint(landmarks.part(47), landmarks.part(46))

        horR_line = cv2.line(frame, Rleft_point, Rright_point, (0, 255, 0), 2)
        verR_line = cv2.line(frame, Rcenter_top, Rcenter_bottom, (0, 255, 0), 2)
        
        #draws a rectangle around the eyes so that it can be easily cropped and prepared for the model to understand
        x=(landmarks.part(17).x,landmarks.part(17).y)
        y=(landmarks.part(15).x,landmarks.part(15).y)
        x1=(landmarks.part(26).x,landmarks.part(26).y)
        y1=(landmarks.part(1).x,landmarks.part(1).y)
        rectangle=cv2.rectangle(frame, x, y, (255,0,0), 2)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    
    if key == ord('w'):
        #takes a picture of the frame
        cv2.imwrite('pic.png',frame)
        img=cv2.imread('pic.png')
        #crops the frame so only the eyes are visible
        cropped_image=img[x[1]:y[1],x[0]:y[0]]
        cv2.imwrite("cropped.jpg", cropped_image)

        categories=['center','right','down','up','left']
        
        renew=prepare('cropped.jpg')
        
        #feeds the prepared image to the model so that it can identify it
        new_model=tf.keras.models.load_model('eye_track.h5')
        prediction=new_model.predict([renew])
        print(categories[int(prediction[0][0])])

    #destroys the window when esc key is pressed
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()


2022-08-23 16:25:11.948006: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


center
center
center
center
center
center
center
center
center
